In [33]:
"""
IMPROVED ML ANALYSIS FOR ECONOMIC COMPLEXITY
- Panel data (all years, not just 2019)
- Temporal train/test split (train: 1995-2015, test: 2016-2019)
- Two model specifications: Resource Curse vs Full Structural
- SHAP for interpretability
- Proper handling of missing data
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import os

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

input_file = "/Users/leoss/Desktop/GitHub/Capstone/MASTER/Master.csv"
production_file = "/Users/leoss/Desktop/GitHub/Capstone/MASTER/NaturalResource.csv"
output_dir = "/Users/leoss/Desktop/Portfolio/Website-/capstone_visualizations/individual_plots/ml"

os.makedirs(output_dir, exist_ok=True)

print("="*70)
print("ML ANALYSIS: PREDICTING ECONOMIC COMPLEXITY")
print("Panel Data with Temporal Train/Test Split")
print("="*70)

# ============================================================================
# 1. LOAD AND PREPARE DATA
# ============================================================================

print("\n1. Loading data...")

df_master = pd.read_csv(input_file)
df_prod = pd.read_csv(production_file)

print(f"   Master data: {len(df_master)} rows, {df_master['Country Code'].nunique()} countries")
print(f"   Years: {df_master['Year'].min()} - {df_master['Year'].max()}")

# Process production data to get resource categories
def categorize_resource(resource):
    if resource == 'Oil': return 'Oil'
    elif resource == 'Natural Gas': return 'Natural Gas'
    elif resource == 'Coal': return 'Coal'
    else: return 'Metals'

df_prod['Resource_Category'] = df_prod['Resource'].apply(categorize_resource)

prod_agg = df_prod.groupby(['Country Name', 'Year', 'Resource_Category'])['Production_TotalValue'].sum().reset_index()
prod_wide = prod_agg.pivot_table(
    index=['Country Name', 'Year'], 
    columns='Resource_Category', 
    values='Production_TotalValue', 
    fill_value=0
).reset_index()
# Clean column names
df_master.columns = df_master.columns.str.replace('—', '-')
# Merge production with master
df = df_master.merge(prod_wide, on=['Country Name', 'Year'], how='left')

# Fill missing production values with 0
for col in ['Oil', 'Natural Gas', 'Coal', 'Metals']:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Calculate GDP-normalized resource values
df['GDP_total'] = df['GDP per capita (constant prices, PPP)'] * df['Population']
for res in ['Oil', 'Natural Gas', 'Coal', 'Metals']:
    if res in df.columns:
        df[f'{res}_GDP_Pct'] = (df[res] / df['GDP_total']) * 100
        df[f'{res}_GDP_Pct'] = df[f'{res}_GDP_Pct'].replace([np.inf, -np.inf], np.nan)

df['Total_Resources_GDP_Pct'] = df[['Oil_GDP_Pct', 'Natural Gas_GDP_Pct', 'Coal_GDP_Pct', 'Metals_GDP_Pct']].sum(axis=1)

print(f"   Merged data: {len(df)} rows")

# ============================================================================
# 2. DEFINE FEATURE SETS
# ============================================================================

print("\n2. Defining feature sets...")

# Target variable
target = 'Economic Complexity Index'

# FEATURE SET A: Resource Curse Model
# Tests: Do natural resources hurt economic complexity, controlling for institutions?
features_resource_curse = [
    # Resource dependency
    'Oil_GDP_Pct',
    'Natural Gas_GDP_Pct', 
    'Coal_GDP_Pct',
    'Metals_GDP_Pct',
    # Human capital (key mediator)
    'Human capital index',
    # Institutions (key for resource curse theory)
    'Rule of law index',
    'Property rights',
    'Political corruption index',
    # Control
    'Landlocked'
]

# FEATURE SET B: Full Structural Model
# Tests: What explains economic complexity variation comprehensively?
features_full = [
    # Resources
    'Oil_GDP_Pct',
    'Natural Gas_GDP_Pct', 
    'Coal_GDP_Pct',
    'Metals_GDP_Pct',
    # Economic structure
    'Manufacturing',
    'Agriculture',
    'Trade (% of GDP)',
    'Gross fixed capital formation, all, Constant prices, Percent of GDP',
    # Human capital
    'Human capital index',
    'Access to electricity (% of population)',
    'Urban population (% of total population)',
    # Institutions
    'Rule of law index',
    'Property rights',
    'Political corruption index',
    'Political stability - estimate',
    # Financial
    'Domestic credit to private sector (% of GDP)',
    'Inflation, consumer prices (annual %)',
    # Controls
    'Landlocked',
]

# Clean feature names for display
feature_names_clean = {
    'Oil_GDP_Pct': 'Oil (% GDP)',
    'Natural Gas_GDP_Pct': 'Natural Gas (% GDP)',
    'Coal_GDP_Pct': 'Coal (% GDP)',
    'Metals_GDP_Pct': 'Metals (% GDP)',
    'Human capital index': 'Human Capital',
    'Rule of law index': 'Rule of Law',
    'Property rights': 'Property Rights',
    'Political corruption index': 'Political Corruption',
    'Political stability - estimate': 'Political Stability',
    'Landlocked': 'Landlocked',
    'Manufacturing': 'Manufacturing (% GDP)',
    'Agriculture': 'Agriculture (% GDP)',
    'Trade (% of GDP)': 'Trade Openness',
    'Gross fixed capital formation, all, Constant prices, Percent of GDP': 'Investment (% GDP)',
    'Access to electricity (% of population)': 'Electricity Access',
    'Urban population (% of total population)': 'Urbanization',
    'Domestic credit to private sector (% of GDP)': 'Private Credit',
    'Inflation, consumer prices (annual %)': 'Inflation'
}

# Define cleaning function
def clean_name(s):
    return (s.replace('—', '-')
             .replace('(', '')
             .replace(')', '')
             .replace('%', 'pct')
             .replace(',', ''))

# Apply to dataframe columns
df_master.columns = [clean_name(c) for c in df_master.columns]

# Apply to feature lists
features_resource_curse = [clean_name(f) for f in features_resource_curse]
features_full = [clean_name(f) for f in features_full]

# Apply to display names dict
feature_names_clean = {clean_name(k): v for k, v in feature_names_clean.items()}

print(f"   Resource Curse Model: {len(features_resource_curse)} features")
print(f"   Full Structural Model: {len(features_full)} features")

# ============================================================================
# 3. PREPARE DATASETS WITH TEMPORAL SPLIT
# ============================================================================

print("\n3. Preparing train/test split...")

# Temporal split: Train on 1995-2015, Test on 2016-2019
TRAIN_END_YEAR = 2015

# Prepare Resource Curse dataset
df_rc = df[['Country Code', 'Country Name', 'Year', target] + features_resource_curse].copy()
df_rc = df_rc.dropna()

train_rc = df_rc[df_rc['Year'] <= TRAIN_END_YEAR]
test_rc = df_rc[df_rc['Year'] > TRAIN_END_YEAR]

X_train_rc = train_rc[features_resource_curse]
y_train_rc = train_rc[target]
X_test_rc = test_rc[features_resource_curse]
y_test_rc = test_rc[target]

print(f"\n   RESOURCE CURSE MODEL:")
print(f"   Train: {len(train_rc)} obs ({train_rc['Year'].min()}-{train_rc['Year'].max()})")
print(f"   Test: {len(test_rc)} obs ({test_rc['Year'].min()}-{test_rc['Year'].max()})")
print(f"   Train countries: {train_rc['Country Code'].nunique()}")
print(f"   Test countries: {test_rc['Country Code'].nunique()}")

# Prepare Full Model dataset
df_full = df[['Country Code', 'Country Name', 'Year', target] + features_full].copy()
df_full = df_full.dropna()

train_full = df_full[df_full['Year'] <= TRAIN_END_YEAR]
test_full = df_full[df_full['Year'] > TRAIN_END_YEAR]

X_train_full = train_full[features_full]
y_train_full = train_full[target]
X_test_full = test_full[features_full]
y_test_full = test_full[target]

print(f"\n   FULL STRUCTURAL MODEL:")
print(f"   Train: {len(train_full)} obs ({train_full['Year'].min()}-{train_full['Year'].max()})")
print(f"   Test: {len(test_full)} obs ({test_full['Year'].min()}-{test_full['Year'].max()})")
print(f"   Train countries: {train_full['Country Code'].nunique()}")
print(f"   Test countries: {test_full['Country Code'].nunique()}")

# Scale features
scaler_rc = StandardScaler()
X_train_rc_scaled = scaler_rc.fit_transform(X_train_rc)
X_test_rc_scaled = scaler_rc.transform(X_test_rc)

scaler_full = StandardScaler()
X_train_full_scaled = scaler_full.fit_transform(X_train_full)
X_test_full_scaled = scaler_full.transform(X_test_full)

# ============================================================================
# 4. TRAIN MODELS
# ============================================================================

print("\n4. Training models...")

def train_and_evaluate(X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled, model_name):
    """Train multiple models and return results"""
    
    results = {}
    
    # Models that need scaling
    models_scaled = {
        'Ridge': Ridge(alpha=1.0, random_state=42),
        'Lasso': Lasso(alpha=0.1, random_state=42),
        'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
    }
    
    # Models that don't need scaling
    models_unscaled = {
        'Random Forest': RandomForestRegressor(
            n_estimators=200, max_depth=10, min_samples_split=10,
            min_samples_leaf=5, random_state=42, n_jobs=-1
        ),
        'Gradient Boosting': GradientBoostingRegressor(
            n_estimators=200, learning_rate=0.05, max_depth=5,
            min_samples_split=10, min_samples_leaf=5, random_state=42
        ),
        'XGBoost': xgb.XGBRegressor(
            n_estimators=200, learning_rate=0.05, max_depth=5,
            min_child_weight=5, subsample=0.8, colsample_bytree=0.8,
            random_state=42, objective='reg:squarederror', verbosity=0
        ),
        'LightGBM': lgb.LGBMRegressor(
            n_estimators=200, learning_rate=0.05, max_depth=5,
            num_leaves=31, min_child_samples=10, subsample=0.8,
            colsample_bytree=0.8, random_state=42, verbose=-1
        )
    }
    
    # Train scaled models
    for name, model in models_scaled.items():
        model.fit(X_train_scaled, y_train)
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
        
        results[name] = {
            'model': model,
            'train_r2': r2_score(y_train, y_pred_train),
            'test_r2': r2_score(y_test, y_pred_test),
            'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test)),
            'test_mae': mean_absolute_error(y_test, y_pred_test),
            'predictions': y_pred_test,
            'scaled': True
        }
        
        # Get coefficients for linear models
        if hasattr(model, 'coef_'):
            results[name]['coefficients'] = model.coef_
    
    # Train unscaled models
    for name, model in models_unscaled.items():
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        results[name] = {
            'model': model,
            'train_r2': r2_score(y_train, y_pred_train),
            'test_r2': r2_score(y_test, y_pred_test),
            'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test)),
            'test_mae': mean_absolute_error(y_test, y_pred_test),
            'predictions': y_pred_test,
            'scaled': False
        }
        
        # Get feature importance for tree models
        if hasattr(model, 'feature_importances_'):
            results[name]['feature_importance'] = model.feature_importances_
    
    return results

# Train Resource Curse models
print("\n   Training Resource Curse models...")
results_rc = train_and_evaluate(
    X_train_rc, X_test_rc, y_train_rc, y_test_rc,
    X_train_rc_scaled, X_test_rc_scaled, "Resource Curse"
)

# Train Full models
print("   Training Full Structural models...")
results_full = train_and_evaluate(
    X_train_full, X_test_full, y_train_full, y_test_full,
    X_train_full_scaled, X_test_full_scaled, "Full Structural"
)

# ============================================================================
# 5. CROSS-VALIDATION
# ============================================================================

print("\n5. Cross-validation (5-fold)...")

kf = KFold(n_splits=5, shuffle=True, random_state=42)

def run_cv(results, X_train, X_train_scaled, y_train):
    cv_results = {}
    for name, res in results.items():
        if res['scaled']:
            X_cv = X_train_scaled
        else:
            X_cv = X_train
        
        cv_scores = cross_val_score(res['model'], X_cv, y_train, cv=kf, scoring='r2')
        cv_results[name] = {
            'mean': cv_scores.mean(),
            'std': cv_scores.std()
        }
    return cv_results

cv_rc = run_cv(results_rc, X_train_rc, X_train_rc_scaled, y_train_rc)
cv_full = run_cv(results_full, X_train_full, X_train_full_scaled, y_train_full)

# ============================================================================
# 6. RESULTS SUMMARY
# ============================================================================

print("\n" + "="*70)
print("MODEL COMPARISON - RESOURCE CURSE MODEL")
print("="*70)

comparison_rc = []
for name, res in results_rc.items():
    comparison_rc.append({
        'Model': name,
        'Train R²': res['train_r2'],
        'Test R²': res['test_r2'],
        'CV R²': cv_rc[name]['mean'],
        'Test RMSE': res['test_rmse'],
        'Overfit': res['train_r2'] - res['test_r2']
    })

df_comp_rc = pd.DataFrame(comparison_rc).sort_values('Test R²', ascending=False)
print(df_comp_rc.to_string(index=False))

print("\n" + "="*70)
print("MODEL COMPARISON - FULL STRUCTURAL MODEL")
print("="*70)

comparison_full = []
for name, res in results_full.items():
    comparison_full.append({
        'Model': name,
        'Train R²': res['train_r2'],
        'Test R²': res['test_r2'],
        'CV R²': cv_full[name]['mean'],
        'Test RMSE': res['test_rmse'],
        'Overfit': res['train_r2'] - res['test_r2']
    })

df_comp_full = pd.DataFrame(comparison_full).sort_values('Test R²', ascending=False)
print(df_comp_full.to_string(index=False))

# ============================================================================
# 7. FEATURE IMPORTANCE / COEFFICIENTS
# ============================================================================

print("\n" + "="*70)
print("FEATURE ANALYSIS")
print("="*70)

# Get best tree model for each
best_tree_rc = df_comp_rc[df_comp_rc['Model'].isin(['Random Forest', 'Gradient Boosting', 'XGBoost', 'LightGBM'])].iloc[0]['Model']
best_tree_full = df_comp_full[df_comp_full['Model'].isin(['Random Forest', 'Gradient Boosting', 'XGBoost', 'LightGBM'])].iloc[0]['Model']

print(f"\n   Best tree model (Resource Curse): {best_tree_rc}")
print(f"   Best tree model (Full): {best_tree_full}")

# Ridge coefficients (interpretable)
print("\n   RIDGE COEFFICIENTS (Resource Curse Model):")
ridge_coefs_rc = pd.DataFrame({
    'Feature': features_resource_curse,
    'Coefficient': results_rc['Ridge']['coefficients']
}).sort_values('Coefficient', key=abs, ascending=False)
print(ridge_coefs_rc.to_string(index=False))

print("\n   RIDGE COEFFICIENTS (Full Model):")
ridge_coefs_full = pd.DataFrame({
    'Feature': features_full,
    'Coefficient': results_full['Ridge']['coefficients']
}).sort_values('Coefficient', key=abs, ascending=False)
print(ridge_coefs_full.to_string(index=False))

# ============================================================================
# 8. VISUALIZATIONS
# ============================================================================

print("\n8. Creating visualizations...")

# ------------------------------
# A. MODEL COMPARISON CHART
# ------------------------------

fig_comp = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Resource Curse Model', 'Full Structural Model')
)

# Resource Curse
df_comp_rc_sorted = df_comp_rc.sort_values('Test R²', ascending=True)
fig_comp.add_trace(
    go.Bar(
        y=df_comp_rc_sorted['Model'],
        x=df_comp_rc_sorted['Test R²'],
        orientation='h',
        marker_color='#002A54',
        text=[f"{x:.3f}" for x in df_comp_rc_sorted['Test R²']],
        textposition='outside',
        name='Resource Curse'
    ),
    row=1, col=1
)

# Full Model
df_comp_full_sorted = df_comp_full.sort_values('Test R²', ascending=True)
fig_comp.add_trace(
    go.Bar(
        y=df_comp_full_sorted['Model'],
        x=df_comp_full_sorted['Test R²'],
        orientation='h',
        marker_color='#E30613',
        text=[f"{x:.3f}" for x in df_comp_full_sorted['Test R²']],
        textposition='outside',
        name='Full Model'
    ),
    row=1, col=2
)

fig_comp.update_layout(
    title=dict(text='Model Performance Comparison (Test R²)', x=0.5, font=dict(size=18)),
    height=500,
    showlegend=False,
    template='plotly_white'
)
fig_comp.update_xaxes(range=[0, 1])

fig_comp.write_html(os.path.join(output_dir, 'ml_model_comparison.html'))
print("   ✓ Model comparison chart saved")

# ------------------------------
# B. FEATURE IMPORTANCE (Tree Models)
# ------------------------------

# Resource Curse Model
importance_rc = pd.DataFrame({
    'Feature': [feature_names_clean.get(f, f) for f in features_resource_curse],
    'Importance': results_rc[best_tree_rc]['feature_importance']
}).sort_values('Importance', ascending=True)

fig_imp_rc = go.Figure()
fig_imp_rc.add_trace(go.Bar(
    y=importance_rc['Feature'],
    x=importance_rc['Importance'],
    orientation='h',
    marker_color='#002A54',
    text=[f"{x:.3f}" for x in importance_rc['Importance']],
    textposition='outside'
))
fig_imp_rc.update_layout(
    title=dict(text=f'Feature Importance - Resource Curse Model ({best_tree_rc})', x=0.5),
    xaxis_title='Importance',
    height=500,
    template='plotly_white',
    margin=dict(l=150)
)
fig_imp_rc.write_html(os.path.join(output_dir, 'ml_feature_importance_resource_curse.html'))

# Full Model
importance_full = pd.DataFrame({
    'Feature': [feature_names_clean.get(f, f) for f in features_full],
    'Importance': results_full[best_tree_full]['feature_importance']
}).sort_values('Importance', ascending=True)

fig_imp_full = go.Figure()
fig_imp_full.add_trace(go.Bar(
    y=importance_full['Feature'],
    x=importance_full['Importance'],
    orientation='h',
    marker_color='#E30613',
    text=[f"{x:.3f}" for x in importance_full['Importance']],
    textposition='outside'
))
fig_imp_full.update_layout(
    title=dict(text=f'Feature Importance - Full Model ({best_tree_full})', x=0.5),
    xaxis_title='Importance',
    height=600,
    template='plotly_white',
    margin=dict(l=150)
)
fig_imp_full.write_html(os.path.join(output_dir, 'ml_feature_importance_full.html'))
print("   ✓ Feature importance charts saved")

# ------------------------------
# C. COEFFICIENT PLOT (Ridge)
# ------------------------------

# Resource Curse coefficients
coef_rc = pd.DataFrame({
    'Feature': [feature_names_clean.get(f, f) for f in features_resource_curse],
    'Coefficient': results_rc['Ridge']['coefficients']
}).sort_values('Coefficient')

fig_coef_rc = go.Figure()
fig_coef_rc.add_trace(go.Bar(
    y=coef_rc['Feature'],
    x=coef_rc['Coefficient'],
    orientation='h',
    marker_color=['#22c55e' if c > 0 else '#ef4444' for c in coef_rc['Coefficient']],
    text=[f"{x:.3f}" for x in coef_rc['Coefficient']],
    textposition='outside'
))
fig_coef_rc.add_vline(x=0, line_dash='dash', line_color='black')
fig_coef_rc.update_layout(
    title=dict(text='Ridge Coefficients - Resource Curse Model<br><sup>Standardized: 1 SD increase in X → β change in ECI</sup>', x=0.5),
    xaxis_title='Coefficient (Standardized)',
    height=500,
    template='plotly_white',
    margin=dict(l=150)
)
fig_coef_rc.write_html(os.path.join(output_dir, 'ml_coefficients_resource_curse.html'))
print("   ✓ Coefficient plots saved")

# ------------------------------
# D. PREDICTED VS ACTUAL
# ------------------------------

best_model_rc = df_comp_rc.iloc[0]['Model']
best_model_full = df_comp_full.iloc[0]['Model']

fig_pred = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        f'Resource Curse Model ({best_model_rc})',
        f'Full Model ({best_model_full})'
    )
)

# Resource Curse
y_pred_rc = results_rc[best_model_rc]['predictions']
fig_pred.add_trace(
    go.Scatter(
        x=y_test_rc, y=y_pred_rc,
        mode='markers',
        marker=dict(color='#002A54', opacity=0.5),
        name='Predictions'
    ),
    row=1, col=1
)

# Full Model
y_pred_full = results_full[best_model_full]['predictions']
fig_pred.add_trace(
    go.Scatter(
        x=y_test_full, y=y_pred_full,
        mode='markers',
        marker=dict(color='#E30613', opacity=0.5),
        name='Predictions'
    ),
    row=1, col=2
)

# Add perfect prediction lines
for col in [1, 2]:
    fig_pred.add_trace(
        go.Scatter(
            x=[-3, 3], y=[-3, 3],
            mode='lines',
            line=dict(color='black', dash='dash'),
            showlegend=False
        ),
        row=1, col=col
    )

fig_pred.update_layout(
    title=dict(text='Predicted vs Actual Economic Complexity Index', x=0.5, font=dict(size=18)),
    height=500,
    showlegend=False,
    template='plotly_white'
)
fig_pred.update_xaxes(title_text='Actual ECI', range=[-3, 3])
fig_pred.update_yaxes(title_text='Predicted ECI', range=[-3, 3])

fig_pred.write_html(os.path.join(output_dir, 'ml_predicted_vs_actual.html'))
print("   ✓ Predicted vs Actual chart saved")

# ------------------------------
# E. RESIDUAL ANALYSIS BY COUNTRY
# ------------------------------

# Add predictions to test data
test_rc_results = test_rc.copy()
test_rc_results['Predicted'] = y_pred_rc
test_rc_results['Residual'] = test_rc_results[target] - test_rc_results['Predicted']

# Countries with largest residuals (model failures)
worst_predictions = test_rc_results.groupby('Country Name').agg({
    'Residual': lambda x: x.abs().mean(),
    target: 'mean',
    'Predicted': 'mean'
}).sort_values('Residual', ascending=False).head(15)

fig_residual = go.Figure()
fig_residual.add_trace(go.Bar(
    y=worst_predictions.index[::-1],
    x=worst_predictions['Residual'][::-1],
    orientation='h',
    marker_color='#f59e0b',
    text=[f"{x:.2f}" for x in worst_predictions['Residual'][::-1]],
    textposition='outside'
))
fig_residual.update_layout(
    title=dict(text='Countries with Largest Prediction Errors (Mean Absolute Residual)', x=0.5),
    xaxis_title='Mean Absolute Residual',
    height=500,
    template='plotly_white',
    margin=dict(l=150)
)
fig_residual.write_html(os.path.join(output_dir, 'ml_residuals_by_country.html'))
print("   ✓ Residual analysis saved")

# ============================================================================
# 9. SHAP ANALYSIS (if available)
# ============================================================================

print("\n9. SHAP analysis...")

try:
    import shap
    
    # Use best tree model for SHAP
    best_model = results_rc[best_tree_rc]['model']
    
    # Create explainer
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X_test_rc)
    
    # Create SHAP summary data
    shap_df = pd.DataFrame({
        'Feature': [feature_names_clean.get(f, f) for f in features_resource_curse],
        'Mean_SHAP': np.abs(shap_values).mean(axis=0)
    }).sort_values('Mean_SHAP', ascending=True)
    
    fig_shap = go.Figure()
    fig_shap.add_trace(go.Bar(
        y=shap_df['Feature'],
        x=shap_df['Mean_SHAP'],
        orientation='h',
        marker_color='#9b59b6',
        text=[f"{x:.3f}" for x in shap_df['Mean_SHAP']],
        textposition='outside'
    ))
    fig_shap.update_layout(
        title=dict(text=f'SHAP Feature Importance - {best_tree_rc}', x=0.5),
        xaxis_title='Mean |SHAP Value|',
        height=500,
        template='plotly_white',
        margin=dict(l=150)
    )
    fig_shap.write_html(os.path.join(output_dir, 'ml_shap_importance.html'))
    print("   ✓ SHAP analysis saved")
    
except ImportError:
    print("   ⚠ SHAP not installed. Run: pip install shap")
except Exception as e:
    print(f"   ⚠ SHAP error: {e}")

# ============================================================================
# 10. SAVE RESULTS
# ============================================================================

print("\n10. Saving results...")

# Save comparison tables
df_comp_rc.to_csv(os.path.join(output_dir, 'ml_comparison_resource_curse.csv'), index=False)
df_comp_full.to_csv(os.path.join(output_dir, 'ml_comparison_full.csv'), index=False)

# Save coefficients
ridge_coefs_rc.to_csv(os.path.join(output_dir, 'ml_ridge_coefficients_rc.csv'), index=False)
ridge_coefs_full.to_csv(os.path.join(output_dir, 'ml_ridge_coefficients_full.csv'), index=False)

# Save feature importance
importance_rc.to_csv(os.path.join(output_dir, 'ml_feature_importance_rc.csv'), index=False)
importance_full.to_csv(os.path.join(output_dir, 'ml_feature_importance_full.csv'), index=False)

# Save predictions
test_rc_results[['Country Code', 'Country Name', 'Year', target, 'Predicted', 'Residual']].to_csv(
    os.path.join(output_dir, 'ml_predictions.csv'), index=False
)

print("   ✓ All results saved")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*70)
print("✅ ML ANALYSIS COMPLETE!")
print("="*70)

print(f"\n📊 RESOURCE CURSE MODEL")
print(f"   Best model: {df_comp_rc.iloc[0]['Model']}")
print(f"   Test R²: {df_comp_rc.iloc[0]['Test R²']:.3f}")
print(f"   Features: {len(features_resource_curse)}")

print(f"\n📊 FULL STRUCTURAL MODEL")
print(f"   Best model: {df_comp_full.iloc[0]['Model']}")
print(f"   Test R²: {df_comp_full.iloc[0]['Test R²']:.3f}")
print(f"   Features: {len(features_full)}")

print(f"\n🔑 KEY FINDINGS (Ridge Coefficients):")
top_positive = ridge_coefs_rc[ridge_coefs_rc['Coefficient'] > 0].nlargest(3, 'Coefficient')
top_negative = ridge_coefs_rc[ridge_coefs_rc['Coefficient'] < 0].nsmallest(3, 'Coefficient')

print(f"   Strongest POSITIVE effects on ECI:")
for _, row in top_positive.iterrows():
    print(f"      {row['Feature']}: +{row['Coefficient']:.3f}")

print(f"   Strongest NEGATIVE effects on ECI:")
for _, row in top_negative.iterrows():
    print(f"      {row['Feature']}: {row['Coefficient']:.3f}")

print(f"\n📁 Outputs saved to: {output_dir}")
print("="*70)

ML ANALYSIS: PREDICTING ECONOMIC COMPLEXITY
Panel Data with Temporal Train/Test Split

1. Loading data...
   Master data: 3150 rows, 126 countries
   Years: 1995 - 2019
   Merged data: 3150 rows

2. Defining feature sets...
   Resource Curse Model: 9 features
   Full Structural Model: 18 features

3. Preparing train/test split...

   RESOURCE CURSE MODEL:
   Train: 2646 obs (1995-2015)
   Test: 504 obs (2016-2019)
   Train countries: 126
   Test countries: 126


KeyError: "['Trade pct of GDP', 'Gross fixed capital formation all Constant prices Percent of GDP', 'Access to electricity pct of population', 'Urban population pct of total population', 'Domestic credit to private sector pct of GDP', 'Inflation consumer prices annual pct'] not in index"